<a href="https://colab.research.google.com/github/mvkumar14/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 2*

---

# Make Features 

- Student should be able to understand the purpose of feature engineering
- Student should be able to work with strings in pandas
- Student should be able to work with dates and times in pandas
- Student should be able to filter a dataframe based on conditions
- Student should be able to modify or create columns of a dataframe using the `.apply()` function


Helpful Links:
- [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428)
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series
- [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)

## Get LendingClub data

[Source](https://www.lendingclub.com/info/download-data.action)

In [1]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-10-01 16:21:28--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [         <=>        ]  21.62M   425KB/s    in 52s     

2019-10-01 16:22:21 (429 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22667400]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [3]:
!head LoanStats_2018Q4.csv

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","op

## Load LendingClub data

pandas documentation
- [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
- [`options.display`](https://pandas.pydata.org/pandas-docs/stable/options.html#available-options)

In [0]:
import pandas as pd
#More information about display options can be seen in the options.display documentation
pd.options.display_Max
df = pd.read_csv('LoanStats_2018Q4.csv',header = 1)
df.head()

In [12]:
df.shape

(128414, 144)

In [0]:
df.int_rate

In [0]:
df.isnull().sum()

In [130]:
# if we look at the last few values in the dataset they are actually just text lines. We can get rid of them
df.tail(3)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_manager
128411,10000.0,10000.0,10000.0,36 months,16.91%,356.08,C,C5,Key Accounts Manager,2 years,RENT,80000.0,Not Verified,2018-10-01,Current,n,other,Other,021xx,MA,17.72,1.0,Sep-2006,0.0,14.0,NaN,17.0,0.0,1942.0,30.8%,31.0,w,7459.19,7459.19,3907.49,3907.49,2540.81,1366.68,0.0,0.0,...,0.0,0.0,73669.0,59194.0,4000.0,67369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,True
128412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
128413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [133]:
#lets use df.drop
df.drop(df.tail(2))

KeyError: ignored

In [132]:
#looking at the documentation for drop it turns out that the error is due to the drop function expecting index values
type(df.tail(3))

pandas.core.frame.DataFrame

In [134]:
#So we can get the index of these from : 
type(df.tail(2).index)

pandas.core.indexes.range.RangeIndex

In [140]:
# here is a proper drop
df.drop(df.tail(2).index).tail()
# We have 2 less than before
print(df.shape)
df.drop(df.tail(2).index).shape

(128414, 141)


(128412, 141)

In [0]:
#now lets change the actual dataframe instead of creating a new one:
df= df.drop(df.tail(2).index)

# you can also do this by setting inplace=True as a parameter within drop() (Inside those parenthesis seperated from the other data with a comma)

In [143]:
#look at the data again, and then we figure out a couple of columns are bad (no data, and we would like to get rid of them)

df = df.drop(['url','member_id','desc','id'],axis=1)

KeyError: ignored

In [0]:
df.head()

## Work with strings

For machine learning, we usually want to replace strings with numbers.

We can get info about which columns have a datatype of "object" (strings)

In [144]:
df.int_rate[:10]

0     14.47%
1     16.14%
2     11.31%
3     12.98%
4     12.98%
5      7.56%
6     17.97%
7     14.47%
8     11.80%
9     17.97%
Name: int_rate, dtype: object

In [145]:
df.dtypes[:10] # note that interest rate is object

loan_amnt          float64
funded_amnt        float64
funded_amnt_inv    float64
term                object
int_rate            object
installment        float64
grade               object
sub_grade           object
emp_title           object
emp_length          object
dtype: object

In [146]:
# we want to turn the interest rate from the following format to float
int_rate = ' 15.02%'
int_rate.strip()

'15.02%'

In [147]:
print(int_rate[0])
# so use int_rate = int_rate.strip() (later)

In [148]:
int_rate.strip().strip('%')

'15.02'

In [149]:
type(int_rate.strip().strip('%'))

str

In [150]:
#This is called casting
float(int_rate.strip().strip('%'))

15.02

In [151]:
type(float(int_rate.strip().strip('%')))

float

### Convert `int_rate`

Define a function to remove percent signs from strings and convert to floats

In [152]:
def int_rate_to_float(int_rate):
  return float(int_rate.strip().strip('%'))
int_rate_to_float(int_rate)

15.02

In [153]:
df['int_rate'].tail()

128405     16.14%
128406     13.56%
128407     15.02%
128408     15.02%
128409     13.56%
Name: int_rate, dtype: object

Apply the function to the `int_rate` column

In [173]:
df['int_rate'] = df['int_rate'].apply(int_rate_to_float)

AttributeError: ignored

### Clean `emp_title`

Look at top 20 titles

In [174]:
df['emp_title'].value_counts(dropna = False,ascending = False)[:10]

NaN                 20946
Teacher              2557
Manager              2395
Registered Nurse     1418
Driver               1258
Supervisor           1160
Truck Driver          920
Rn                    834
Office Manager        805
Sales                 803
Name: emp_title, dtype: int64

How often is `emp_title` null?

In [175]:
#or you could do dropna =False for value_counts
df['emp_title'].isnull().sum()

20946

In [176]:
df['emp_title'].value_counts().reset_index().shape

(34901, 2)

Clean the title and handle missing values

In [177]:
import numpy as np
type(np.NaN)

float

Write a function to clean up the titles 

In [178]:
examples = ['owner','Supervisor','Project Manager',np.NaN]

def clean_title(title):
  #Strip whitespace, and then title casing
  if isinstance(title,str):
    return title.strip().title()

[clean_title(item) for item in examples]

['Owner', 'Supervisor', 'Project Manager', None]

Now we can try to use this on a column of a dataframe


In [0]:
df['emp_title'] = df['emp_title'].apply(clean_title)

In [180]:
len(df['emp_title'].value_counts())
#this is down from before by about 25%

34901

### Create `emp_title_manager`

pandas documentation: [`str.contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html)

In [181]:
df['emp_title_manager'] = df['emp_title'].str.contains("Manager")
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_manager
0,20000.0,20000.0,20000.0,36 months,14.47,688.13,C,C2,Bus Driver,4 years,OWN,52000.0,Source Verified,2018-12-01,Current,n,debt_consolidation,Debt consolidation,681xx,NE,30.65,1.0,Jun-1979,2.0,15.0,NaN,6.0,0.0,15048.0,73%,22.0,w,15777.74,15777.74,6104.740000,6104.74,4222.26,1882.48,0.0,0.0,...,0.0,0.0,61699.0,33157.0,20700.0,40999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,False
1,25000.0,25000.0,25000.0,60 months,16.14,609.82,C,C4,Production Manager,5 years,MORTGAGE,45000.0,Not Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,703xx,LA,37.09,0.0,Sep-2003,0.0,NaN,NaN,7.0,0.0,8901.0,36.8%,21.0,w,0.00,0.00,26653.167580,26653.17,25000.00,1653.17,0.0,0.0,...,0.0,0.0,73683.0,49524.0,18600.0,49483.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,True
2,26500.0,26500.0,26500.0,60 months,11.31,580.28,B,B3,Compliance Director,10+ years,MORTGAGE,134000.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,747xx,OK,18.91,0.0,Dec-2006,2.0,NaN,NaN,17.0,0.0,43640.0,65.4%,37.0,w,23410.70,23410.70,5197.540000,5197.54,3089.30,2108.24,0.0,0.0,...,0.0,0.0,430403.0,107975.0,49300.0,88875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,False
3,10400.0,10400.0,10400.0,36 months,12.98,350.32,B,B5,Program Support Assistant,10+ years,MORTGAGE,56099.0,Source Verified,2018-12-01,Current,n,credit_card,Credit card refinancing,800xx,CO,20.92,0.0,Jul-2013,2.0,32.0,67.0,8.0,1.0,1669.0,9.8%,10.0,w,8164.57,8164.57,3141.630000,3141.63,2235.43,906.20,0.0,0.0,...,1.0,0.0,60709.0,39564.0,17000.0,43709.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,False
4,35000.0,35000.0,35000.0,60 months,12.98,796.00,B,B5,Sr. Vice President -Risk Manager,7 years,MORTGAGE,211646.0,Source Verified,2018-12-01,Fully Paid,n,debt_consolidation,Debt consolidation,797xx,TX,15.09,0.0,Sep-1985,0.0,77.0,NaN,13.0,0.0,55944.0,74.9%,39.0,w,0.00,0.00,36266.741552,36266.74,35000.00,1266.74,0.0,0.0,...,0.0,0.0,516204.0,82514.0,59700.0,51904.0,88364.0,Sep-1985,0.0,6.0,19.0,53.4,2.0,29.0,0.0,0.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,True


## Data Frame Filtering Example:
subset of dataframe based on certain conditions

In [0]:
# make new dfs where the second df is a subset of the original
condition = (df['emp_title_manager'] == True)

# here is my new data frame:
manager = df[condition]

# This means that everyone here is a manager:
manager.head()


In [0]:
#"oppsite of manager"
condition = (df['emp_title_manager'] == False)
# note that the we are comparing lists of values using & this is a bitwise opperator (doing a comparison for every item in the list)
peon = df[condition]
print(peon.shape)
peon.head()

In [190]:
# now we can compare an object 
manager['int_rate'].mean()

12.760350092276147

In [191]:
peon['int_rate'].mean()

12.945226438051943

WE can make exploratory visualizations for the subsets because we filtered

In [0]:
import seaborn as sns
sns.distplot(peon['installment'])
sns.distplot(manager['installment']);

## Work with dates

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

In [0]:
# here are all the columns
[col for col in df]

In [106]:
# one good thing about this dataset is that there is a suffix of _d for colums that are date times

# here is a list column that will give us just the datetimes
[col for col in df if col.endswith('_d')]f

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [110]:
df.dtypes[10:15] # note that issue date here is an object

home_ownership          object
annual_inc             float64
verification_status     object
issue_d                 object
loan_status             object
dtype: object

In [112]:
df['issue_d'][:10]

0    Dec-2018
1    Dec-2018
2    Dec-2018
3    Dec-2018
4    Dec-2018
5    Dec-2018
6    Dec-2018
7    Dec-2018
8    Dec-2018
9    Dec-2018
Name: issue_d, dtype: object

In [116]:
df['issue_d'] = pd.to_datetime(df['issue_d'],infer_datetime_format = True)
df['issue_d'].head()

0   2018-12-01
1   2018-12-01
2   2018-12-01
3   2018-12-01
4   2018-12-01
Name: issue_d, dtype: datetime64[ns]

In [117]:
df['issue_d'][:10] # note that now the dtype is datetime64[ns]

0   2018-12-01
1   2018-12-01
2   2018-12-01
3   2018-12-01
4   2018-12-01
5   2018-12-01
6   2018-12-01
7   2018-12-01
8   2018-12-01
9   2018-12-01
Name: issue_d, dtype: datetime64[ns]

In [118]:
df['issue_d'].head().values

array(['2018-12-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [122]:
# we can parse out specific portions of the date:
df['issue_d'].dt.month[:10]

0    12.0
1    12.0
2    12.0
3    12.0
4    12.0
5    12.0
6    12.0
7    12.0
8    12.0
9    12.0
Name: issue_d, dtype: float64